<a href="https://colab.research.google.com/github/ElioRame/ProgrammingAssignment2/blob/master/FinalAssignmentPALS0039.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:

with open("/content/pg74.txt") as TomSawyer:
  Book1 = TomSawyer.readlines()
with open("/content/pg84.txt") as Frankenstein:
  Book2 = Frankenstein.readlines()
with open("/content/pg345.txt") as Dracula:
  Book3 = Dracula.readlines()
with open("/content/pg1342.txt") as PrideandPrejudice:
  Book4 = PrideandPrejudice.readlines()
with open("/content/pg1727.txt") as Odissey:
  Book5 = Odissey.readlines()
with open("/content/pg2701.txt") as MobyDick:
  Book6 = MobyDick.readlines()
with open("/content/pg3207.txt") as Leviathan:
  Book7 = Leviathan.readlines()

data = Book1 + Book2 + Book3 + Book4 + Book6 + Book7
print(data[:50])

['\ufeffThe Project Gutenberg eBook of The Adventures of Tom Sawyer, Complete\n', '    \n', 'This ebook is for the use of anyone anywhere in the United States and\n', 'most other parts of the world at no cost and with almost no restrictions\n', 'whatsoever. You may copy it, give it away or re-use it under the terms\n', 'of the Project Gutenberg License included with this ebook or online\n', 'at www.gutenberg.org. If you are not located in the United States,\n', 'you will have to check the laws of the country where you are located\n', 'before using this eBook.\n', '\n', 'Title: The Adventures of Tom Sawyer, Complete\n', '\n', 'Author: Mark Twain\n', '\n', 'Release date: July 1, 2004 [eBook #74]\n', '                Most recently updated: August 9, 2023\n', '\n', 'Language: English\n', '\n', 'Credits: David Widger\n', '\n', '\n', '*** START OF THE PROJECT GUTENBERG EBOOK THE ADVENTURES OF TOM SAWYER, COMPLETE ***\n', '\n', '\n', '\n', '\n', 'THE ADVENTURES OF TOM SAWYER\n', '\n', '\n', '

In [66]:
!pip install -U nltk>=3.7.0

import nltk
nltk.download("reuters")
nltk.download("punkt")
nltk.download('punkt_tab')
from nltk.corpus import reuters
from nltk.lm import Vocabulary
from nltk.util import pad_sequence, ngrams
from nltk.tokenize import RegexpTokenizer
from nltk.lm.preprocessing import flatten
from nltk.lm.models import Laplace

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Embedding, Flatten, LSTM, TimeDistributed
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [67]:
import string
import collections
import operator


letters = []
padded_letters = []

for text in data:
  lower_text = [word.lower() for word in text]
  for words in lower_text:
   tokenizer = RegexpTokenizer(r'\w+')
   tokenised_word = tokenizer.tokenize(words)
   for word in tokenised_word:
    for letter in word:
      letters.append(letter)



   #for punct in string.punctuation:
    #letters = list(map(lambda x: " " if x == punct else x, tokenised_word))
     #for punct in string.punctuation:
      #text_punct = list(map(lambda x: " " if x == punct else x, letter))
      #letters.append(letter)



#def tokenize(text):
  #text_lower = [word.lower() for word in str(text)]
  #for punct in string.punctuation:
    #text_punct = list(map(lambda x: " " if x == punct else x, text_lower))
  #return text_punct

#tokenized_data = tokenize(data)
#for words in tokenized_data:
  #for letter in words:
    #letters.append(letter)





padded_lower_s = list(pad_sequence(letters,
                                     n=2,
                                     pad_left=True,
                                     left_pad_symbol="<s>",
                                     pad_right=True,
                                     right_pad_symbol="</s>"))
padded_letters.append(padded_lower_s)


# 2. Flatten and create finite vocabulary
words = list(flatten(padded_letters))




vocab = Vocabulary(words, unk_cutoff=10)

# 3. Partition into train and test
test_words = words[:10000]
train_words = words[10000:]

#Inspect
print("Length of corpus:", len(words))
print("Length of vocabulary:", len(vocab))
print("Unique words in corpus:", len(set(words)))

v = [word for word in vocab]
print(v)

Length of corpus: 3845425
Length of vocabulary: 42
Unique words in corpus: 59
['t', 'h', 'e', 'p', 'r', 'o', 'j', 'c', 'g', 'u', 'n', 'b', 'k', 'f', 'a', 'd', 'v', 's', 'm', 'w', 'y', 'l', 'i', '1', '2', '0', '4', '7', '9', '3', 'z', 'x', 'q', '8', '6', '_', 'é', 'ê', '5', 'æ', 'è', '<UNK>']


In [68]:
#Create mapping
word_to_index = {w: i for i, w in enumerate(vocab)}

#Map data
train_indices = [word_to_index[vocab.lookup(w)] for w in train_words]
test_indices = [word_to_index[vocab.lookup(w)] for w in test_words]

#Inspect
print(train_indices[:25])
print(test_indices[:25])




[22, 17, 11, 2, 0, 0, 2, 4, 10, 20, 5, 9, 21, 5, 5, 12, 35, 0, 1, 22, 17, 35, 0, 22, 18]
[41, 0, 1, 2, 3, 4, 5, 6, 2, 7, 0, 8, 9, 0, 2, 10, 11, 2, 4, 8, 2, 11, 5, 5, 12]


In [69]:
seqlen = 100

def prepare_sequences(words, unk_index, seqlen=seqlen):
  trunc_length = len(words) % seqlen
  X = np.array(words)[trunc_length:].reshape((-1, seqlen))
  y = np.array(words + [unk_index])[trunc_length + 1:].reshape((-1, seqlen))
  return X, y

Xtrain, ytrain = prepare_sequences(train_indices, word_to_index["<UNK>"])
Xtest, ytest = prepare_sequences(test_indices, word_to_index["<UNK>"])

#Inspect
print(Xtrain.shape, ytrain.shape)
print(Xtest.shape, ytest.shape)

(38354, 100) (38354, 100)
(100, 100) (100, 100)


In [70]:
def perplexity(y_true, y_pred):
  cross_entropy = tf.losses.sparse_categorical_crossentropy(y_true, y_pred)
  perplexity = tf.exp(tf.reduce_mean(cross_entropy))
  return perplexity

def build_model(input_width=len(vocab), input_length=seqlen, output_width=len(vocab), learning_rate=0.01):
  model = Sequential()
  model.add(Embedding(input_dim=input_width, output_dim=64, input_length=input_length))
  model.add(LSTM(32, return_sequences=True, activation='tanh'))
  model.add(Dense(32, activation='tanh'))
  model.add(TimeDistributed(Dense(output_width, activation='softmax')));
  model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=[perplexity])
  return model

model = build_model()

#Inspect
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_10 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_19 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_8              │ ?                      │   0 (unbuilt) │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
training_info = model.fit(Xtrain,ytrain, batch_size=64, validation_split=0.10, epochs=40, callbacks=EarlyStopping(monitor="val_loss", patience=5))

Epoch 1/40
540/540 ━━━━━━━━━━━━━━━━━━━━ 71s 131ms/step - loss: 2.0249 - perplexity: 7.5772 - val_loss: 1.9859 - val_perplexity: 7.2909
Epoch 2/40
258/540 ━━━━━━━━━━━━━━━━━━━━ 33s 121ms/step - loss: 2.0291 - perplexity: 7.6093

In [ ]:
def plot_training_history(training_info):
  fig, axs = plt.subplots(1, 2, figsize=(16, 5))
  axs[0].plot(training_info.history['loss'], label="training set")
  axs[0].plot(training_info.history['val_loss'], label="validation set")
  axs[0].set_xlabel("Epoch")
  axs[0].set_ylabel("Loss")
  axs[0].grid(True)
  axs[0].legend()
  try:
    axs[1].plot(training_info.history['perplexity'], label="training set")
    axs[1].plot(training_info.history['val_perplexity'], label="validation set")
    axs[1].set_xlabel("Epoch")
    axs[1].set_ylabel("Perplexity")
    axs[1].grid(True)
    axs[1].legend()
  except:
    pass
  plt.show()

plot_training_history(training_info)

In [ ]:
test_loss, test_perplexity = model.evaluate(Xtest, ytest, verbose=0)
print("Test loss:", test_loss)
print("Test perplexity:", test_perplexity)

In [ ]:
from scipy.stats import rv_discrete

index_to_word = {i: w for w, i in word_to_index.items()}

# Choose seed text and create padded index sequence
seed_text = "it is a fact that"
for w in seed_text.split():
  tokenizer.tokenize(w)
  for word in w:
    for letters in word:
     seq = [word_to_index[l] for l in letters]
padded_seq = pad_sequences([seq], maxlen=seqlen, padding='post')

# Sample word sequence
for i in range(seqlen - len(seq)):
  full_prediction = model.predict(padded_seq, verbose=0)
  next_one_letter = full_prediction[0][len(seq) + i, :]
  next_two_letters = next_one_letter[0][len(seq) + i, :]
  next_three_letters = next_two_letters[0][len(seq) + i, :]
  next_three_letters_prediction = [next_one_letter, next_two_letters, next_three_letters]
  dpd = rv_discrete(values=(list(range(len(next_three_letters_prediction))), next_three_letters_prediction))
  padded_seq[0][len(seq) + i] = dpd.rvs(size=1)

# Print word sequence
sampled_words = []
for word_index in padded_seq[0]:
  sampled_words.append(index_to_word[word_index])
print(" ".join(sampled_words))